# Exploration of using the Modified Hazardous Situation ODP

> 

A modified Hazardous Situation ODP (Ontology Design Pattern) is a refined version of the existing pattern that enhances its ability to support risk assessment and mitigation planning. This modification focuses on breaking down aspects of exposure to a hazard into individual components, allowing for a more granular understanding of the impact on different objects and people.

Here's a more detailed explanation:
Existing Hazardous Situation ODP:
The original pattern effectively addresses core competency questions related to hazard exposure.

The modified pattern expands on the original by:
- Breaking down exposure components: This allows for a more precise analysis of how different objects and individuals are affected by the same hazard. 
- Supporting risk assessment and mitigation: The modified pattern enables a more comprehensive approach to identifying and addressing potential hazards. 
- Considering individual susceptibility: The modification acknowledges that different entities (objects, people, etc.) may be impacted differently by the same hazard due to their unique characteristics. 

Key Benefit:
- The modified pattern helps in identifying and addressing potential hazards more effectively by considering the specific context of each situation. 


![MH-ODP](./hazard/hazard-odp.png)

Turning the Hazardous‑Situation stack into a “conscience loop”

(all labels correspond to files or modules you already have in CogitareLink)

┌────────────── sensors / world model ─────────────┐
│  depth cam, IMU, PLC tags …                      │
└──────┬────────────────────────────────────────────┘
       │ 1. fresh state triples  (Layer 3 data.jsonld)
┌──────▼───────────────┐
│  LLM Planner         │  (Chain‑of‑Thought + skill tools)
│  • decomposes goal    │
│  • proposes plan P    │  → JSON‑LD action graph
└──────┬───────────────┘
       │ 2. *pre‑flight conscience check*
┌──────▼───────────────────────────┐
│  CogitareLink sandbox            │  (single tool: `reason_over`)
│  pass A – SHACL                  │
│    uses `haz-shapes.ttl`         │  ➜ flag risk, emit violation
│  pass B – EYE (optional)         │
│    uses `haz-logic.n3`           │  ➜ obligations, CTD fixes
└──────┬───────────────────────────┘
       │ 3. NL summary + prov IRI
┌──────▼─────────────────┐
│  LLM Moral Governor    │
│  • reads violations / duties
│  • decides:            │
│      a) reject plan           (risk un‑mitigated)  
│      b) revise plan           (insert mitigating steps)  
│      c) approve & explain     (compliant)
└──────┬─────────────────┘
       │ 4. approved plan P′
┌──────▼──────────┐
│  Low‑level exec │  (controllers, ROS2, PLC)
└─────────────────┘


⸻

1.  What the “conscience” actually does

Stage	Concrete check / synthesis	Backing artifact
Detect	“Any participant’s exposure > susceptibility?”	RiskEvalShape (SHACL rule)
Judge	Map violation → obligation (CTD)	DeonticHazardRules (N3)
Explain	Produce NL sentence:  \n“Plan would expose Pump‑A to 150 psi (> 120 psi limit).  Obligation: throttle flow or abort.”	summary from reason_over, plus prov:Activity proof
Guide	Feed obligation back to planner (insert_mitigation() function or a fresh tool call)	handled by LLM Governor


⸻

2.  Wiring in code (pseudo‑Python)

def conscience_check(plan_jsonld: str) -> bool:
    # 1. Run SHACL
    res1 = reason_over(jsonld=plan_jsonld,
                       shapes_turtle=HAZ_SHAPE_TTL)
    if not res1['violations']:
        return True                     # safe, no risk

    # 2. Run EYE for CTD duties
    res2 = reason_over(jsonld=plan_jsonld,
                       n3_rules=HAZ_N3)
    duties = res2['derived'].filter(dl.obliges)

    # 3. Decide
    if duties:
        planner.insert_steps(duties)    # revise plan
        return conscience_check(planner.current_plan())
    else:
        return False                    # reject – no remedy

One recursive call is usually enough because CTD rules introduce an explicit mitigation step.

⸻

3.  Fast‑path vs. slow‑path
	•	Fast‑path (no risk): only SHACL fires → single tool call, < 10 ms.
	•	Slow‑path (risk): SHACL + EYE + 1 plan rewrite → ~40 ms on a laptop.
This fits well inside a 10 Hz control‑loop envelope for mobile robots and is trivial latency for industrial arms that run at 250 ms cycles.

⸻

4.  Why the split SHACL → EYE matters

Reason	Impact on embodied agent
Streaming‑friendly	SHACL validation can run incrementally as sensor facts arrive; no need to wait for full plan.
Explain‑first	Violations come with human‑readable messages, so the robot can speak “Potential overpressure detected.” before deciding on mitigation.
Formality when needed	EYE only spins up when real conflict appears, avoiding proof‑engine overhead on every cycle.
CTD logic	Lets the system repair a risky plan instead of blindly cancelling — key for mission continuity.


⸻

5.  Example dialogue loop

User: “Robot, swap the reactor filters.”
Robot: “Hold on. Performing risk check…”
Conscience output: Violation: Exposure to corrosive gas > threshold. Obligation: wear viton seal gloves.
Robot: “I must equip viton gloves first. Proceed?”
User: “Yes.”
Robot: executes revised plan with mitigation step.

All reasoning‑layer provenance is logged; if auditors ask why the robot insisted on gloves, you point them to the prov:Activity proof containing the violated shape and fired CTD rule.

⸻

6.  Extensible beyond hazards

Identical pattern supports:
	•	Privacy conscience: SHACL detects PII leakage → N3 obliges anonymisation.
	•	Ethical alignment: SHACL detects human discomfort cues → N3 imposes obligation to re‑phrase utterance.
	•	Energy budget: SHACL flags battery below threshold → N3 obliges return‑to‑dock.

Swap in different Layer 2a/2b bundles; the micro‑kernel stays untouched.

⸻

Bottom line

By sandwiching fast, declarative SHACL risk detection with expressive N3 deontic rules—and looping the result back to the LLM planner—you get a lightweight conscience that:
	1.	Catches hazards before action execution.
	2.	Offers concrete, plan‑level remedies (CTD).
	3.	Maintains a provable audit trail.

All with nothing more exotic than two reason_over calls and a small governing callback around your existing planner.